<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/File_share_45_Arrival_predictions_corrected_with_Headway_calculation_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

15/07/2024

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
import time

def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (BST)'] = pd.to_datetime(df['Expected Arrival (BST)'])  # Convert to datetime
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Gap'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['2_Gap'] = (df['Gap'] * 2).round(2)
        df['Gap_Sq'] = (df['Gap'] * df['Gap']).round(2)
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

def main():
    line_id = input("Enter Line ID: ")
    stop_point_id = input("Enter Stop Point ID: ")
    direction = input("Enter Direction (inbound/outbound): ")

    cumulative_df = pd.DataFrame(columns=[
        'Line', 'Vehicle ID', 'Stop Point', 'Direction',
        'Expected Arrival (BST)', 'Expected Arrival (HM)',
        'Gap', '2_Gap', 'Gap_Sq'
    ])

    while True:
        arrival_predictions_df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)

        if arrival_predictions_df is not None and not arrival_predictions_df.empty:
            current_hour = datetime.now(pytz.timezone('Europe/London')).hour
            next_hour = (current_hour + 1) % 24

            for _, row in arrival_predictions_df.iterrows():
                vehicle_id = row['Vehicle ID']
                expected_hour = row['Expected Arrival (BST)'].hour
                if (vehicle_id in cumulative_df['Vehicle ID'].values and
                    expected_hour in [current_hour, next_hour]):
                    cumulative_df.loc[(cumulative_df['Vehicle ID'] == vehicle_id) &
                                      (cumulative_df['Expected Arrival (BST)'].dt.hour.isin([current_hour, next_hour])), :] = row.values
                else:
                    cumulative_df = pd.concat([cumulative_df, row.to_frame().T]).reset_index(drop=True)

            # Recalculate headway for cumulative_df
            cumulative_df = cumulative_df.sort_values(by='Expected Arrival (BST)', ascending=True)
            cumulative_df['Expected Arrival (BST)'] = pd.to_datetime(cumulative_df['Expected Arrival (BST)'])  # Ensure datetime format
            cumulative_df['Expected Arrival (HM)'] = pd.to_datetime(cumulative_df['Expected Arrival (HM)'], format='%H:%M')

            # Identify the first imp_row
            cumulative_df['Gap'] = cumulative_df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
            cumulative_df['Gap'] = cumulative_df.apply(
                lambda row: 0 if row.name == 0 or row['Expected Arrival (BST)'].hour != cumulative_df.loc[row.name - 1, 'Expected Arrival (BST)'].hour else row['Gap'],
                axis=1
            )

            cumulative_df['2_Gap'] = (cumulative_df['Gap'] * 2).round(2)
            cumulative_df['Gap_Sq'] = (cumulative_df['Gap'] * cumulative_df['Gap']).round(2)

            # Calculate the number of buses observed in the current hour
            num_buses_observed = cumulative_df[cumulative_df['Expected Arrival (BST)'].dt.hour == current_hour].shape[0]

            print(f"\nArrival Predictions for stop point {stop_point_id} ({station_name}):")
            print(arrival_predictions_df.to_string(index=False))
            print("\nCumulative DataFrame:")
            print(cumulative_df.to_string(index=False))
            print(f"\nNumber of buses observed in the current hour: {num_buses_observed}")
        else:
            print("No arrival predictions available.")

        print("Refreshing data in 30 seconds...\n")
        time.sleep(30)  # Wait for 30 seconds before fetching data again

if __name__ == "__main__":
    main()



Arrival Predictions for stop point 490011107G (Poplar / All Saints Church):
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
  D7    LX61DBO 490011107G  outbound    2024-07-16 12:32:37   1900-01-01 12:32:00  0.0    0.0     0.0
  D7    LX11BJV 490011107G  outbound    2024-07-16 12:45:40   1900-01-01 12:45:00 13.0   26.0   169.0
  D7    LX61DBU 490011107G  outbound    2024-07-16 12:57:34   1900-01-01 12:57:00 12.0   24.0   144.0

Cumulative DataFrame:
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
  D7    LX61DBO 490011107G  outbound    2024-07-16 12:32:37   1900-01-01 12:32:00  0.0    0.0     0.0
  D7    LX11BJV 490011107G  outbound    2024-07-16 12:45:40   1900-01-01 12:45:00 13.0   26.0   169.0
  D7    LX61DBU 490011107G  outbound    2024-07-16 12:57:34   1900-01-01 12:57:00 12.0   24.0   144.0

Number of buses observed in the current hour: 3
Refreshing data in 30 seconds...


